# 5.7 수업 내용
# 지난 수업 selector + 오늘 수업 Crawling 

# CSS SELECTOR

Selector => Tag, ID(#아이디), CLASS(.클래스명)<br>
[name = 'asdf']<br>
[name ^= 'asdfasd']<br>
[name $= 'asdfasd']<br>
div p => find_all(div 찾고, 자손중에 p)<br>
div > p => find_all(recursive = False)<br>
div 찾고, 자식 p<br>
div + p => find(div).find_next_sibling(P)<br>
div태그 중 class가 article_view인 태그 탐색 body = soup.select('div.article_view')[0]<br>


In [1]:
import requests
from bs4 import BeautifulSoup

# html 문서를 다운로드하는 함수

In [2]:
from urllib import parse

header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}

def getDownload(url, params={}, retries=3):
    resp = None
    
    try:
        resp = requests.get(url, params=params, headers=header)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and retries > 0:
            print(retries)
            resp = getDownload(url, params, retries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
            
    return resp

# 다음 검색창

In [5]:
html = getDownload("https://search.daum.net/search", {"q":"박보영"})
daum = BeautifulSoup(html.text, "lxml")

In [6]:
[_["href"] for _ in daum.select("div.wrap_tit > a")]

['http://v.media.daum.net/v/20190508081336464?f=o',
 'http://v.media.daum.net/v/20190507140359069?f=o',
 'http://v.media.daum.net/v/20190507185002021?f=o',
 'http://v.media.daum.net/v/20190507175700537?f=o',
 'http://papa0717.tistory.com/223',
 'http://dudrms606.tistory.com/367',
 'http://adam24eve.tistory.com/858',
 'http://k3k2y.tistory.com/35',
 'https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81',
 'https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81%20%28%EB%B2%95%EC%A1%B0%EC%9D%B8%29',
 'http://cafe.daum.net/subdued20club/ReHf/2280152?q=%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://cafe.daum.net/subdued20club/ReHf/2282606?q=%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://cafe.daum.net/ASMONACOFC/gAVU/1243818?q=%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://cafe.daum.net/ok1221/9Zdf/1524913?q=%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://tip.daum.net/answer/54503481?q=%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://tip.daum.net/answer/54767339?q=%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://tip.daum.net/answer/550

# 네이버 실시간 순위

In [8]:
html = getDownload("https://www.naver.com/")
naver = BeautifulSoup(html.text, "lxml")

In [9]:
[_.get_text() for _ in naver.select('.area_hotkeyword .ah_k ')]

['한지성',
 '이인영',
 '인천공항고속도로 사고',
 '여배우 사망',
 '에어서울',
 '어버이날 문구',
 '류현진 완봉',
 '유인석',
 '한선교',
 '김태년',
 '배우 한지성',
 '완봉승',
 '오마이걸',
 '더불어민주당 원내대표',
 '밀정',
 '강은비 하나경',
 '박한별',
 '승리 구속',
 '원펀맨 2기 5화',
 '김관영',
 '한지성',
 '이인영',
 '인천공항고속도로 사고',
 '여배우 사망',
 '에어서울',
 '어버이날 문구',
 '류현진 완봉',
 '유인석',
 '한선교',
 '김태년',
 '배우 한지성',
 '완봉승',
 '오마이걸',
 '더불어민주당 원내대표',
 '밀정',
 '강은비 하나경',
 '박한별',
 '승리 구속',
 '원펀맨 2기 5화',
 '김관영']

In [11]:
for tag in naver.select('.ah_list.PM_CL_realtimeKeyword_list_base .ah_k'):
    print(tag.text)

한지성
이인영
인천공항고속도로 사고
여배우 사망
에어서울
어버이날 문구
류현진 완봉
유인석
한선교
김태년
배우 한지성
완봉승
오마이걸
더불어민주당 원내대표
밀정
강은비 하나경
박한별
승리 구속
원펀맨 2기 5화
김관영


# Crawling

단어들 인덱스할 것임<br>
scrapy vs beautifulsoup 차이<br>
scrapy는 프레임워크 bs는 라이브러리<br>
페이지 한번 방문할 때 마다 링크를 축적해가며 다시 세팅하여 크롤한다<br>

# Crawling 예제 페이지!

In [13]:
seed = "http://example.webscraping.com/places/default/index"
html = getDownload(seed)
dom = BeautifulSoup(html.content, 'lxml')

In [16]:
links = [_["href"] for _ in dom.select("a")]
links

['#',
 '/places/default/user/register?_next=/places/default/index',
 '/places/default/user/login?_next=/places/default/index',
 '/places/default/index',
 '/places/default/search',
 '/places/default/view/Afghanistan-1',
 '/places/default/view/Aland-Islands-2',
 '/places/default/view/Albania-3',
 '/places/default/view/Algeria-4',
 '/places/default/view/American-Samoa-5',
 '/places/default/view/Andorra-6',
 '/places/default/view/Angola-7',
 '/places/default/view/Anguilla-8',
 '/places/default/view/Antarctica-9',
 '/places/default/view/Antigua-and-Barbuda-10',
 '/places/default/index/1']

### #이면 자바스크립트 시작기호? 무시해야함 -> url 정규화 필요

In [17]:
for link in links:
    # # , /, http, https 가 올 것임
    if link[0] == "/":
        print(requests.compat.urljoin(seed, link))

http://example.webscraping.com/places/default/user/register?_next=/places/default/index
http://example.webscraping.com/places/default/user/login?_next=/places/default/index
http://example.webscraping.com/places/default/index
http://example.webscraping.com/places/default/search
http://example.webscraping.com/places/default/view/Afghanistan-1
http://example.webscraping.com/places/default/view/Aland-Islands-2
http://example.webscraping.com/places/default/view/Albania-3
http://example.webscraping.com/places/default/view/Algeria-4
http://example.webscraping.com/places/default/view/American-Samoa-5
http://example.webscraping.com/places/default/view/Andorra-6
http://example.webscraping.com/places/default/view/Angola-7
http://example.webscraping.com/places/default/view/Anguilla-8
http://example.webscraping.com/places/default/view/Antarctica-9
http://example.webscraping.com/places/default/view/Antigua-and-Barbuda-10
http://example.webscraping.com/places/default/index/1


# *** 핵심! url 축적하면서 돌리는 함수

In [26]:
threshold = 3

def urlExtractor(seed, limit):

    if limit > threshold:
        return []
    
    html = getDownload(seed)
    dom = BeautifulSoup(html.content, "lxml")
    
    links = [_["href"] for _ in dom.select("a") if _.has_attr('href')]
    
    unseen = list()
    
    for link in links:
        if len(link) > 1 and link[0] == "/":
            newLink = requests.compat.urljoin(seed, link)            
            if newLink not in unseen:
                unseen.append((newLink, limit + 1))
#             else:
#                 print("Skipped2", link)
        # 외부로 나가는 주소
        elif link.startswith("http"):
            unseen.append((link, limit + 1))
#         else:    
#             print("Skipped1", link)
            
    return unseen

# 구글링크 traversal
# 구글 검색창으로 테스트 depth(limit)는 3까지

In [27]:
html = getDownload('https://www.google.com/search', {'q':'박보영'})
dom = BeautifulSoup(html.content, 'lxml')
queue = [(_['href'], 0) for _ in dom.select('.r > a')]
queue

[('https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', 0),
 ('https://namu.wiki/w/%EB%B0%95%EB%B3%B4%EC%98%81', 0),
 ('https://www.msn.com/ko-kr/entertainment/news/%EB%B0%95%EB%B3%B4%EC%98%81-tvn-%EB%93%9C%EB%9D%BC%EB%A7%88-%EC%96%B4%EB%B9%84%EC%8A%A4-%EC%97%AC%EC%A3%BC%EC%9D%B8%EA%B3%B5/ar-BBLZzVh',
  0),
 ('https://twitter.com/hashtag/%EB%B0%95%EB%B3%B4%EC%98%81', 0),
 ('https://news.joins.com/article/22895953', 0),
 ('https://www.mk.co.kr/star/hot-issues/view/2019/05/300046/', 0),
 ('https://www.mk.co.kr/star/hot-issues/view/2019/05/293931/', 0),
 ('https://www.hankyung.com/entertainment/article/201905030904H', 0)]

In [ ]:
import time
import random

seen = list()

# 큐가 더이상 없을때까지 돌리기
while queue:
    link = queue.pop(0)
    seen.append(link[0])
#     time.sleep(random.randint(1,3))
    links = urlExtractor(link[0], link[1])
    
    queueURL = [_[0] for _ in queue]
    queue.extend([_ for _ in links if _[0] not in queueURL \
                     and _ not in seen ])
    
    print("{0} {1} [{2}]".format(">"*link[1], link[0], len(links)))
#     html = download(link)
#     dom = BeautifulSoup(html.content, 'lxml')
    # links 는 unseen에 겹치면 안되고 seen에 있으면 안됨
#     queue.extend([_ for _ in links if _ not in queue \
#                   and _ in seen])

In [33]:
print("depth 3으로 해서 쌓인 링크 개수 :", len(queue))

depth 3으로 해서 쌓인 링크 개수 : 29790


### url이 어떻게 분리되는지 보여주기

In [39]:
requests.compat.urlparse(queue[0][0])

ParseResult(scheme='https', netloc='ko.wikipedia.org', path='/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%9D%B8%EC%9A%A9_%EC%98%A4%EB%A5%98_%EB%8F%84%EC%9B%80%EB%A7%90', params='', query='', fragment='bad_date')

# 심화 : 네이버 블로그만 크롤링하기

In [44]:
requests.compat.urlparse('https://blog.naver.com/search.naver?where=post&sm=tab_jum&query=%EB%B0%95%EB%B3%B4%EC%98%81')[1]

'blog.naver.com'

In [45]:
html = getDownload('https://search.naver.com/search.naver', {'query':'박보영', "where":"post"})
dom = BeautifulSoup(html.text, 'lxml')
queue = [(_["href"], 0) for _ in \
            dom.select("dt > a.sh_blog_title")]

In [46]:
queue

[('https://blog.naver.com/js2y86?Redirect=Log&logNo=221530397718', 0),
 ('https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950', 0),
 ('https://blog.naver.com/lyj0088?Redirect=Log&logNo=221530708253', 0),
 ('https://blog.naver.com/shkwon1128?Redirect=Log&logNo=221530738046', 0),
 ('https://komartin.blog.me/221530734469', 0),
 ('https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195', 0),
 ('https://blog.naver.com/yustock1004?Redirect=Log&logNo=221531740045', 0),
 ('https://blog.naver.com/comint3?Redirect=Log&logNo=221532102016', 0),
 ('https://blog.naver.com/theboss005?Redirect=Log&logNo=221531781511', 0),
 ('https://blog.naver.com/insusul?Redirect=Log&logNo=221531749084', 0)]

# 네이버 블로그만 크롤하게 수정한 코드

In [47]:
threshold = 3

def urlExtractor(seed, limit):
    if limit > threshold:
        return []
    html = getDownload(seed)
    dom = BeautifulSoup(html.content, "lxml")

    # iframe을 한번 더 들어가서 crawl해야 함
    iframe = dom.select_one("iframe")
    
    if not iframe or not iframe.has_attr("src"):
        return []
    
    html = getDownload(
                requests.compat.urljoin(seed, iframe["src"]))
    dom = BeautifulSoup(html.text, 'lxml')
    links = [_["href"] for _ in dom.select("a") if _.has_attr('href')]
    
    unseen = list()
    
    for link in links:
        if len(link) > 1 and link[0] == "/":
            newLink = requests.compat.urljoin(seed, link)            
            if newLink not in unseen:
                unseen.append((newLink, limit + 1))
            else:
                print("Skipped2", link)
        # 외부로 나가는 주소
        elif link.startswith("http"):
            unseen.append((link, limit + 1))
#         else:    
#             print("Skipped1", link)
            
    return unseen

In [ ]:
while queue:
    link = queue.pop(0)
    seen.append(link[0])
    
#     time.sleep(random.randint(1,3))
    links = urlExtractor(link[0], link[1])
#     print(links)
    queueURL = [_[0] for _ in queue]
    queue.extend([_ for _ in links if _[0] not in queueURL \
                     and _[0] not in seen])
    
    print("{0} {1} [{2}]".format(">"*link[1], link[0], len(links)))

# 번외

In [ ]:
html = Gownload("https://arxiv.org/pdf/1708.02709.pdf")
with open("1708.02709.pdf", "wb") as fp:
    fp.write(html.content)